## git clone & Load Dataset

In [1]:
# UNetPlusPlus Reference git clone
! git clone https://github.com/4uiiurz1/pytorch-nested-unet.git

Cloning into 'pytorch-nested-unet'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 77 (delta 19), reused 16 (delta 16), pack-reused 53
Unpacking objects: 100% (77/77), done.


In [2]:
# Dataset Download
!gdown 11aCGZSG3dZrYVgglYFktlXlAAsmiJr8s

Downloading...
From: https://drive.google.com/uc?id=11aCGZSG3dZrYVgglYFktlXlAAsmiJr8s
To: /content/Multi_512x512_BalancedLabel_minsize.zip
100% 173M/173M [00:02<00:00, 80.7MB/s]


In [3]:
# requirements.txt install을 위해 경로설정
%cd /content/pytorch-nested-unet

/content/pytorch-nested-unet


In [ ]:
!pip install -r requirements.txt

In [ ]:
# /content에 Dataset 압축풀기
!unzip /content/Multi_512x512_BalancedLabel_minsize.zip -d /content

## 모델링에서 요구하는 데이터셋 디렉토리 만들기
inputs 폴더 내에 images, masks 폴더가 있어야 합니다

masks/0/* 형태로, masks폴더 내 0번폴더에 mask data가 들어가야 합니다

mkdir을 통해 폴더를 만들어준 뒤, gdown한 데이터셋 안에서 각자 본인이 맡은 라벨번호를 위 설명처럼 만들어주세요

mkdir 실행 후 그 content에서 경로따라가면 폴더 생성된거 확인 가능

In [6]:
# git clone해온 파일 내부의 inputs폴더에 dataset폴더 생성
!mkdir inputs/dataset

In [7]:
# 위에서 만들어준 dataset폴더 안에 masks폴더 생성
!mkdir inputs/dataset/masks

In [8]:
# 현재 경로 확인
!pwd

/content/pytorch-nested-unet


In [9]:
# 현재 경로에 들어있는 파일들 확인
!ls

archs.py    LICENSE	models		       README.md	 utils.py
dataset.py  losses.py	outputs		       requirements.txt  val.py
inputs	    metrics.py	preprocess_dsb2018.py  train.py


In [10]:
!mv /content/Multi_512x512_BalancedLabel_minsize/images /content/pytorch-nested-unet/inputs/dataset
!mv /content/Multi_512x512_BalancedLabel_minsize/labels /content/Multi_512x512_BalancedLabel_minsize/masks
!mv /content/Multi_512x512_BalancedLabel_minsize/masks /content/pytorch-nested-unet/inputs/dataset

In [11]:
!rm -rf /content/pytorch-nested-unet/inputs/dataset/.ipynb_checkpoints
!rm -rf /content/pytorch-nested-unet/inputs/.gitkeep

## 모델 학습시키기
python 명령어를 통해 train.py파일 실행

In [23]:
!python train.py --dataset /content/pytorch-nested-unet/inputs/dataset  -b 4 \
                 --epochs 50 \
                 --optimizer AdamW \
                 --input_w 512 \
                 --input_h 512 \
                 --num_classes 2

--------------------
name: /content/pytorch-nested-unet/inputs/dataset_NestedUNet_woDS
epochs: 50
batch_size: 4
arch: NestedUNet
deep_supervision: False
input_channels: 3
num_classes: 2
input_w: 512
input_h: 512
loss: BCEDiceLoss
dataset: /content/pytorch-nested-unet/inputs/dataset
img_ext: .png
mask_ext: .png
optimizer: AdamW
lr: 0.001
momentum: 0.9
weight_decay: 0.0001
nesterov: False
scheduler: CosineAnnealingLR
min_lr: 1e-05
factor: 0.1
patience: 2
milestones: 1,2
gamma: 0.6666666666666666
early_stopping: -1
num_workers: 4
--------------------
=> creating model NestedUNet
Traceback (most recent call last):
  File "train.py", line 356, in <module>
    main()
  File "train.py", line 252, in main
    train_img_ids, val_img_ids = train_test_split(img_ids, test_size=0.2, random_state=41)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py", line 2421, in train_test_split
    n_samples, test_size, train_size, default_test_size=0.25
  File "/usr/local/lib/pyth

In [ ]:
# 경로내에있는 파일개수 확인, os.listdir('mask폴더경로')를 사용하여 mask data 개수확인
import os
len(os.listdir('/content/pytorch-nested-unet/inputs/dataset/masks/0'))

287

## torch version확인 ( 1.12.0 인지 확인해주세요)

In [14]:
import torch
print(torch.__version__)

1.12.1+cu113


## AdamW 추가

In [ ]:
## train.py ' # optimizer ' argument ( line 70 ) 아래 코드로 바꿔주기
    # # optimizer
    # parser.add_argument('--optimizer', default='SGD',
    #                     choices=['Adam', 'SGD', 'AdamW'],
    #                     help='loss: ' +
    #                     ' | '.join(['Adam', 'SGD', 'AdamW']) +
    #                     ' (default: Adam)')

In [ ]:
## train.py line 230 에 아래 코드 추가
    # elif config['optimizer'] == 'AdamW':
    #     optimizer = optim.AdamW(params, lr=config['lr'], betas=(0.9, 0.999), eps=1e-08, weight_decay=config['weight_decay'], amsgrad=False, maximize=False)

## archs.py

In [ ]:
## class UNet 에서 생성자함수 파라미터 ( line 29) 에 deep_supervision=False 추가
    # def __init__(self, num_classes, input_channels=3, deep_supervision=False, **kwargs):